In [2]:
# Install dependencies
!pip install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install gradio


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-e3n1e6n0
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-e3n1e6n0
  Resolved https://github.com/facebookresearch/detectron2.git to commit b1c43ffbc995426a9a6b5c667730091a384e0fa4
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 M

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [1]:
import torch
import torchvision.transforms as T
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.models.detection import maskrcnn_resnet50_fpn
import numpy as np
import gradio as gr

In [2]:
def refine_masks(mask, threshold=0.5):
    """Refine the masks based on threshold and remove small regions."""
    mask = mask > threshold
    mask = mask.float()
    return mask

def remove_background_advanced(image):
    # Load the image
    image = Image.fromarray(image).convert("RGB")

    # Define the transformation
    transform = T.Compose([
        T.ToTensor(),
        T.Resize((1024, 1024))  # Increasing the resolution for better accuracy
    ])
    image_tensor = transform(image)

    # Load the pre-trained Mask R-CNN model
    model = maskrcnn_resnet50_fpn(pretrained=True)
    model.eval()

    # Move image to the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    image_tensor = image_tensor.to(device)
    model.to(device)

    # Perform inference
    with torch.no_grad():
        prediction = model([image_tensor])

    # Extract the most confident mask
    scores = prediction[0]['scores']
    masks = prediction[0]['masks']
    highest_score_index = scores.argmax()
    mask = masks[highest_score_index, 0]

    # Refine the mask
    mask = refine_masks(mask, threshold=0.8)  # Adjust threshold for better results

    # Apply the refined mask to the image
    masked_image = image_tensor * mask

    # Convert the tensor back to an image
    return T.ToPILImage()(masked_image.cpu()).convert("RGB")

In [3]:
# Gradio interface
def gradio_interface(image):
    result_image = remove_background_advanced(image)
    return result_image

In [4]:
# Create Gradio interface
iface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Image(type="numpy"),
    outputs=gr.Image(type="pil"),
    title="Background Removal with Mask R-CNN",
    description="Upload an image to remove its background using Mask R-CNN."
)

In [5]:
# Launch the interface
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f38dcf9ea80ec6725a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [6]:
import torch

def calculate_metrics(pred_mask, true_mask):
    """Calculate accuracy, precision, recall, and F1-score."""
    # Calculate True Positives, False Positives, True Negatives, False Negatives
    tp = torch.logical_and(pred_mask, true_mask).sum().item()
    fp = torch.logical_and(pred_mask, torch.logical_not(true_mask)).sum().item()
    tn = torch.logical_and(torch.logical_not(pred_mask), torch.logical_not(true_mask)).sum().item()
    fn = torch.logical_and(torch.logical_not(pred_mask), true_mask).sum().item()

    # Calculate Accuracy
    accuracy = (tp + tn) / (tp + fp + tn + fn)

    # Calculate Precision
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0

    # Calculate Recall
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0

    # Calculate F1 Score
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return accuracy, precision, recall, f1_score

# Example placeholders for predicted_mask and true_mask as torch tensors of the same size
predicted_mask = torch.tensor([[1, 0], [0, 1]])  # Placeholder tensor
true_mask = torch.tensor([[1, 0], [0, 0]])  # Placeholder tensor

accuracy, precision, recall, f1_score = calculate_metrics(predicted_mask, true_mask)
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")


Accuracy: 0.7500
Precision: 0.5000
Recall: 1.0000
F1 Score: 0.6667
